## Data processing
----
In this notebook, we will compute different quantities which will be useful to the training of our artificial neural network as well as for plotting. 
We will also change the units of AME data to be sure everything is standardized. 
In the end, we will create a merged .csv file which will contain of the useful data for the next part of this project.


In [1]:
#Libraries for data processing
import pandas as pd
import numpy as np

### We will calculate some more data for DZ10

-Binding Energy

-S1p & S2p

-S2p & S2n

Data is already in MeV

#### A and Binding Energy

In [2]:
#Reading DZ10 data as dz_data
dz_data = pd.read_csv("2_processed_data/dz_data.csv", sep=";")

#We create a new column in pandas dataframe "dz_data" containing mass number
dz_data["A"] = dz_data["Z"] + dz_data["N"]

#We create a new column containing Binding Energy (we multiply by A)
dz_data["dz_BE"] = dz_data["dz_BE/A"] * dz_data["A"]

#### Computation of S1n and S1p


In [3]:
dz_data.sort_values(by=['Z'], ascending=True)
dz_data['dz_S1n'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(1)


dz_data = dz_data.sort_values(by=['N','A'], ascending=True)
dz_data['dz_S1p'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(1)

#### Computation of S2n and S2p

In [4]:
#Data already well sorted by the computation of S1p
#dz_data = dz_data.sort_values(by=['N','A'], ascending=True)
dz_data['dz_S2p'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(2)

dz_data = dz_data.sort_values(by=['Z','A'], ascending=True)
dz_data['dz_S2n'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(2)

### We will calculate some data for AME

-Binding energy 

-S2n & S2p

We will also drop some data that we finally don't use

#### Binding Energy

In [5]:
ame_data = pd.read_csv("2_processed_data/mass_data.csv", sep=";")

ame_data['ame_BE'] = ame_data['ame_BE/A'] * ame_data['A']

#### Computation of S1n, S1p, S2n and S2p

In [6]:
ame_data = ame_data.sort_values(by=['N','Z'], ascending=True)
ame_data['ame_S2p'] = ame_data['ame_BE'] - ame_data['ame_BE'].shift(2)
ame_data['ame_S1p'] = ame_data['ame_BE'] - ame_data['ame_BE'].shift(1)

ame_data = ame_data.sort_values(by=['Z','N'], ascending=True)
ame_data['ame_S2n'] = ame_data['ame_BE'] - ame_data['ame_BE'].shift(2)
ame_data['ame_S1n'] = ame_data['ame_BE'] - ame_data['ame_BE'].shift(1)

### From these two documents we will merge all the data we need in one dataframe and .csv file

#### First of all, we will change units and calculate some quantities that may be useful to train our artificial neural networks

In [7]:
merged_data=pd.merge(dz_data, ame_data, on=["Z", "N", "A"])

#Change of units
#AME file gives energies in keV
#DZ10 file gives energies in MeV
#We want MeV at the end
merged_data["ame_BE/A"] = merged_data["ame_BE/A"]/1000
merged_data["ame_ME"] = merged_data["ame_ME"]/1000
merged_data["ame_BE"] = merged_data["ame_BE"]/1000
merged_data["ame_S2p"] = merged_data["ame_S2p"]/1000
merged_data["ame_S2n"] = merged_data["ame_S2n"]/1000
merged_data["ame_S1p"] = merged_data["ame_S1p"]/1000
merged_data["ame_S1n"] = merged_data["ame_S1n"]/1000

#We compute the difference in binding energy AME - DZ10
merged_data["Difference_BE_DZ_AME"] = merged_data["ame_BE"]- merged_data["dz_BE"]

#### We compute some quantities related to the Liquid Drop Model


In [8]:
alphaV=15.49 
alphaS=17.23
alphaC=0.697
alphaA=23.285
alphaP=12

merged_data["Surf"] = np.power(merged_data["A"],2/3)
merged_data["Asym"] = ( (merged_data["N"]-merged_data["Z"])**2 ) / merged_data["A"]
merged_data["Coul"] = ( merged_data["Z"]*(merged_data["Z"]-1) ) / np.power(merged_data["A"],1/3)
merged_data["Pair"] = np.power(merged_data["A"],-1/2)
merged_data["Z_parity"] = np.power(-1,merged_data["Z"])
merged_data["N_parity"] = np.power(-1,merged_data["N"])

merged_data["ldm_BE"]=alphaV * merged_data["A"] - alphaS * merged_data["Surf"] - alphaC * merged_data["Coul"] - alphaA * merged_data["Asym"]

#### Then we compute others quantities that give more physics terms such as the parity for N and Z, and their distance with respect to magic numbers

In [9]:
merged_data["Z_parity"] = np.power(-1,merged_data["Z"])
merged_data["N_parity"] = np.power(-1,merged_data["N"])

magic_numbers = [2, 8, 20, 28, 50, 82, 126, 184]

#Adding two columns for the distance with respect to magic numbers
merged_data["Z_distance"] = None
merged_data["N_distance"] = None

#Compute the distance to magic numbers
for i, row in merged_data.iterrows():
    z = row["Z"]
    n = row["N"]
    merged_data.at[i, "Z_distance"] = min([abs(z - m) for m in magic_numbers])
    merged_data.at[i, "N_distance"] = min([abs(n - m) for m in magic_numbers])

In [10]:
#We save this merged dataframe to .csv
merged_data.to_csv("2_processed_data/merged_data.csv",sep=";", index=False)